Anggota Kelompok 4
<br><br>
No&nbsp;Nama &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;NIM
<br>
1   &ensp;Achmad Wibawa           &emsp;&emsp;&emsp;&emsp;2041720174<br>
3   &ensp;Ahmad Abid              &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2041720179<br>
4   &ensp;Ahmad Thariq Ramadhan   2041720200<br>
10  &nbsp;Dewangga Putra          &emsp;&emsp;&emsp;&emsp;2042720222<br>
17  &nbsp;Maulana Rosandy         &emsp;&emsp;&emsp;2041720120<br>
18  &nbsp;Moch Fajrul Falah       &emsp;&emsp;&emsp;2041720070<br>

In [53]:
import numpy as np
import pandas as pd

data = pd.read_csv('colle_scrap.csv')
data.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1607163989674590209,1607163989674590209,2022-12-26 06:58:49 SE Asia Standard Time,2022-12-26,06:58:49,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1607163313649246208,1607163313649246208,2022-12-26 06:56:08 SE Asia Standard Time,2022-12-26,06:56:08,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1607162707584876544,1607162707584876544,2022-12-26 06:53:43 SE Asia Standard Time,2022-12-26,06:53:43,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1607162057639755777,1607162057639755777,2022-12-26 06:51:08 SE Asia Standard Time,2022-12-26,06:51:08,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1607161879671222272,1607161879671222272,2022-12-26 06:50:26 SE Asia Standard Time,2022-12-26,06:50:26,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [54]:
print(data.duplicated(subset=['tweet']).sum())
data.drop_duplicates(subset=['tweet'], inplace = True)

24


In [55]:
pd.isnull(data['tweet']).sum()

0

In [56]:
import re 
data1 = data.copy()

rm_rt_url = lambda x: re.sub('(\[cm\]|\[CM\]|\[Cm\]|\[cM\]) | (@[A-Za-z0–9\w]+) | (@\w+:) | (\w+:\/\/\S+) | (www.\S+)',' ',x)
rm_punct = lambda x: re.sub('\W', ' ', x)

data1['tweet'] = data.tweet.map(rm_rt_url).map(rm_punct)
data1['tweet'].head()

0     dimana tempat kalian berkeluh kesah soal kehi...
1     temen2 pernah gini gak  tiba tiba muncul some...
2     guys  ada yang mau bantu like tugasku gak di ...
3            kampus swasta tuh lg libir nataru ga sih 
4     cara chat dosbim buat follow up masalah bab1 ...
Name: tweet, dtype: object

In [57]:
data1['tweet'].str.lower()
data1['tweet'].head()

0     dimana tempat kalian berkeluh kesah soal kehi...
1     temen2 pernah gini gak  tiba tiba muncul some...
2     guys  ada yang mau bantu like tugasku gak di ...
3            kampus swasta tuh lg libir nataru ga sih 
4     cara chat dosbim buat follow up masalah bab1 ...
Name: tweet, dtype: object

Tokenizing menggunakan NLTK

In [58]:
from nltk.tokenize import TweetTokenizer
token = TweetTokenizer()

tweet = data1['tweet'].apply(token.tokenize)
tweet.head()

0    [dimana, tempat, kalian, berkeluh, kesah, soal...
1    [temen, 2, pernah, gini, gak, tiba, tiba, munc...
2    [guys, ada, yang, mau, bantu, like, tugasku, g...
3    [kampus, swasta, tuh, lg, libir, nataru, ga, sih]
4    [cara, chat, dosbim, buat, follow, up, masalah...
Name: tweet, dtype: object

Stemming dengan SnowballStemmer dan Sastrawi

Snowballstemmer lebih cepat 10x

In [59]:
from snowballstemmer import stemmer
snow_stem = stemmer('indonesian')


def snow_stemming(text):
    stem_text = [snow_stem.stemWord(word) for word in text]
    return stem_text

tweet = tweet.apply(lambda x: snow_stemming(x))

tweet.head()

0    [mana, tempat, kali, keluh, kesah, soal, hidup...
1    [temen, 2, pernah, gini, gak, tiba, tiba, munc...
2    [guys, ada, yang, mau, bantu, like, tugas, gak...
3    [kampus, swasta, tuh, lg, libir, nataru, ga, sih]
4    [cara, chat, dosbim, buat, follow, up, masa, b...
Name: tweet, dtype: object

In [60]:
from textblob import TextBlob

for row in data1.itertuples():
    tweet = data1.at[row[0], 'tweet']
#run sentiment using TextBlob
    analysis = TextBlob(tweet)
    data1.at[row[0], 'polarity'] = analysis.sentiment[0]
    data1.at[row[0], 'subjectivity'] = analysis.sentiment[1]
#Create Positive / negative column depending on polarity
    if analysis.sentiment[0]>0:
        data1.at[row[0], 'sentiment'] = "Positive"
    elif analysis.sentiment[0]<0:
        data1.at[row[0], 'sentiment'] = "Negative"
    else:
        data1.at[row[0], 'sentiment'] = "Neutral"
    

data1[['tweet','sentiment','polarity','subjectivity']].head()


,tweet,sentiment,polarity,subjectivity
0,dimana tempat kalian berkeluh kesah soal kehi...,Neutral,0.0,0.0
1,temen2 pernah gini gak tiba tiba muncul some...,Negative,-0.5,0.9
2,guys ada yang mau bantu like tugasku gak di ...,Neutral,0.0,0.0
3,kampus swasta tuh lg libir nataru ga sih,Neutral,0.0,0.0
4,cara chat dosbim buat follow up masalah bab1 ...,Neutral,0.0,0.0


In [61]:
counts = data1['sentiment'].value_counts()

counts

Neutral     4591
Positive     268
Negative     117
Name: sentiment, dtype: int64

In [62]:
from sklearn.preprocessing import LabelEncoder

X = data1['tweet'].values
y = data1['sentiment'].values

# Encode Label
encoder = LabelEncoder()

y = encoder.fit_transform(y)
print(y.shape)


(4976,)


In [63]:
# Split Train Test
# Split dulu untuk menghindari leaking information
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [64]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

## 1st Experiment
mnb = MultinomialNB()

tfidf_vect = TfidfVectorizer()

X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)
mnb_tf = MultinomialNB()

mnb_tf.fit(X_train_tfidf, y_train)

# Predict train
y_tf_train = mnb_tf.predict(X_train_tfidf)
acc_tf_train = accuracy_score(y_train, y_tf_train)

# Predict test
y_tf_test = mnb_tf.predict(X_test_tfidf)
acc_tf_test = accuracy_score(y_test, y_tf_test)

# Print Hasil
print('==========')
print(f'Akurasi fitur TF-IDF (training): {acc_tf_train}')
print(f'Akurasi fitur TF-IDF (testing): {acc_tf_test}')

Akurasi fitur TF-IDF (training): 0.921105527638191
Akurasi fitur TF-IDF (testing): 0.929718875502008


In [65]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc = SVC(kernel="linear")

svc_tf = SVC()

svc_tf.fit(X_train_tfidf, y_train)

# Predict train
y_tf_train = svc_tf.predict(X_train_tfidf)
acc_tf_train = accuracy_score(y_train, y_tf_train)

# Predict test
y_tf_test = svc_tf.predict(X_test_tfidf)
acc_tf_test = accuracy_score(y_test, y_tf_test)

# Print Hasil
print('==========')
print(f'Akurasi fitur TF-IDF (training): {acc_tf_train}')
print(f'Akurasi fitur TF-IDF (testing): {acc_tf_test}')

Akurasi fitur TF-IDF (training): 0.9608040201005025
Akurasi fitur TF-IDF (testing): 0.9337349397590361


In [68]:
import pickle

# Save the model to a file
with open("mnb.pkl", "wb") as f:
    pickle.dump(mnb_tf, f)
    
with open("svc.pkl", "wb") as f:
    pickle.dump(svc_tf, f)